# AECTech2025 Hackathon - SIMON

In [1]:
from pathlib import Path
from PIL import Image
import numpy as np
import json
import plotly.io as pio
import plotly.express as px
import pandas as pd

pio.renderers.default = "vscode"

In [517]:
DATA_DIR = Path("/Users/vincentmai/Projects/AECTech_2025/data/tt_conference_room_02")
KEYFRAME_DIR = Path(DATA_DIR / "keyframes")
CAMERA_DIR = Path(KEYFRAME_DIR / "corrected_cameras")
IMAGE_DIR = Path(KEYFRAME_DIR / "corrected_images")
DEPTH_DIR = Path(KEYFRAME_DIR / "depth")
DETECTION_OUTPUT_DIR = Path(DATA_DIR / "yolo_detection")

## Parsing Polycam Camera Position and Orientation

In [518]:
def unitize(v):
    return v / np.linalg.norm(v)

In [519]:
def parse_camera_data(camera_data):
    rotation = []
    for i in range(3):
        rotation.append([camera_data[f"t_{i}{j}"] for j in range(4)])
    rotation.append([0.0, 0.0, 0.0, 1.0])
    principal_point = ([camera_data["cx"], camera_data["cy"]])
    focal_length = ([camera_data["fx"], camera_data["fy"]]) 
    return {
        "camera_to_world": np.array(rotation),
        "principal_point": principal_point,
        "focal_length": focal_length,
    }

Polycam export less images than true camera position, need to filter based on actual exported images

In [520]:
exported_images_names = set([path.stem for path in IMAGE_DIR.rglob("*.jpg")])

In [521]:
camera_data = dict()
positions = []
for file_path in CAMERA_DIR.rglob("*.json"):
    stem = file_path.stem
    if stem not in exported_images_names: continue
    with open(file_path, 'r') as f:
        raw_data = json.load(f)
        parsed = parse_camera_data(raw_data)
        camera_data[stem] = parsed

## Parsing Frame Detection Result

In [522]:
def parse_bbox_center(bbox):
    xmin, ymin, width, height = bbox
    return xmin + width / 2, ymin + height / 2

def parse_detection_attr(detection_result, confidence_threshold=0.5):
    parsed = []
    for frame_id, frame_results in detection_result["detections"].items():
        for result in frame_results:
            if result["confidence"] < confidence_threshold: continue 
            xformed = result.copy()
            xformed["frame_loc"] = parse_bbox_center(result["bbox"])
            parsed.append(xformed)
    return parsed

def parse_unique_detection_attr(detection_result):
    parsed = dict()
    for frame_id, frame_results in detection_result["detections"].items():
        for result in frame_results:
            label = result["class_name"]
            confidence = parsed.get(label, {"confidence": -1})["confidence"]
            if result["confidence"] > confidence:
                parsed[label] = dict()
                xformed = result.copy()
                xformed["frame_loc"] = parse_bbox_center(result["bbox"])
                parsed[label]["confidence"] = result["confidence"]
                parsed[label]["data"] = (xformed)
    return [item["data"] for item in parsed.values()]

Exported Images Dim: 1024 x 768
Depth Maps Dim: 256 x 192
Depth Map unit is in mm

In [523]:
def pixel_to_world(u, v, depth, fx, fy, cx, cy, camera_to_world_matrix):
    """
    Maps a 2D pixel coordinate (and its depth) to a 3D world coordinate.

    Parameters:
    - u (float): The horizontal pixel coordinate (column).
    - v (float): The vertical pixel coordinate (row).
    - depth (float): The depth (Z-value) of the pixel in the camera's coordinate system.
    - fx (float): The horizontal focal length (in pixels).
    - fy (float): The vertical focal length (in pixels).
    - cx (float): The horizontal principal point (in pixels).
    - cy (float): The vertical principal point (in pixels).
    - camera_to_world_matrix (np.ndarray): A 4x4 NumPy array representing the
                                            camera's pose (extrinsics), transforming
                                            points from camera space to world space.

    Returns:
    - np.ndarray: A 3D NumPy array (x, y, z) representing the point in world space.
    """

    # --- Step 1: Un-project from pixel space to camera space ---

    # The formulas to go from 3D camera space (Xc, Yc, Zc) to pixels (u, v) are:
    # u = (Xc * fx / Zc) + cx
    # v = (Yc * fy / Zc) + cy

    # We can reverse these formulas to solve for Xc and Yc, given u, v, and Zc (depth).
    # Xc * fx / Zc = u - cx  =>  Xc = (u - cx) * Zc / fx
    # Yc * fy / Zc = v - cy  =>  Yc = (v - cy) * Zc / fy

    # Zc is simply the given depth
    Zc = depth
    Xc = (u - cx) * Zc / fx
    Yc = (v - cy) * Zc / fy

    # This gives us the 3D point in the camera's local coordinate system
    P_camera = np.array([Xc, Yc, Zc])

    # --- Step 2: Transform from camera space to world space ---

    # To use the 4x4 transformation matrix, we need to convert our 3D point
    # to homogeneous coordinates by adding a 1 (P_camera_homogeneous = [Xc, Yc, Zc, 1])
    P_camera_homogeneous = np.array([P_camera[0], P_camera[1], P_camera[2], 1.0])

    # Apply the transformation by matrix-multiplying with the camera-to-world matrix
    # P_world_homogeneous = CameraToWorld_Matrix @ P_camera_homogeneous
    P_world_homogeneous = camera_to_world_matrix @ P_camera_homogeneous

    # --- Step 3: Convert back from homogeneous coordinates ---
    # Divide the (x, y, z) components by the 4th component (w)
    # to get the final 3D point in world space.
    if P_world_homogeneous[3] != 0:
        Xw = P_world_homogeneous[0] / P_world_homogeneous[3]
        Yw = P_world_homogeneous[1] / P_world_homogeneous[3]
        Zw = P_world_homogeneous[2] / P_world_homogeneous[3]
        
        P_world = np.array([Xw, Yw, Zw])
        return P_world
    else:
        # This case should not happen with standard transformations
        raise ValueError("Homogeneous coordinate 'w' is zero, cannot convert to 3D.")

In [530]:
def get_depth(depth_map, loc):
    DEPTH_UNIT = 1 / 2_000 # mm
    SCALE_FACTOR = 4
    y, x = loc
    return depth_map[int(x) // SCALE_FACTOR, int(y) // SCALE_FACTOR] * DEPTH_UNIT
    
def get_obj_spatial_loc(detection_attr, camera_data, depth_map):
    u, v = detection_attr["frame_loc"]
    cx, cy = camera_data["principal_point"]
    fx, fy = camera_data["focal_length"]
    obj_depth = get_depth(depth_map, (u, v))
    return pixel_to_world(u, v, obj_depth, fx, fy, cx, cy, camera_data["camera_to_world"])

In [531]:
depth_maps = dict()
for file_path in DEPTH_DIR.rglob("*.png"):
    stem = file_path.stem
    if stem not in exported_images_names: continue
    with Image.open(file_path, 'r') as img:
        img_arr = np.array(img)
        depth_maps[stem] = img_arr

In [532]:
detection_data = dict()
for file_path in DETECTION_OUTPUT_DIR.rglob("*.json"):
    with open(file_path, 'r') as f:
        data = json.load(f)
        for parsed in parse_detection_attr(data):
            detection_data[parsed["frame_id"]] = parsed

In [533]:
obj_locations = []
obj_labels = []
obj_data = dict()
for id, detection in detection_data.items():
    camera = camera_data[str(id)]
    depth_map = depth_maps[str(id)]
    depth = get_depth(depth_map, detection["frame_loc"])
    if depth > 3: continue
    obj_loc = get_obj_spatial_loc(detection, camera, depth_map)
    obj_labels.append(detection["class_name"])
    camera_loc = camera["camera_to_world"][:3, 3]
    obj_locations.append(obj_loc)
    dist_to_camera = obj_loc - camera_loc
    direction = np.zeros(16).reshape(4,4)
    direction[:3, 3] = dist_to_camera
    obj_data[id] = {
        "direction": direction.tolist(),
        "label": detection["class_name"],
        "confidence": detection["confidence"],
    }

In [534]:
with open(DATA_DIR / "label_positions.json", 'w') as f:
    json.dump(obj_data, f, indent=4)

In [ ]:
df_camera = pd.DataFrame(
    [camera["camera_to_world"][:3, 3] for camera in camera_data.values()], 
    columns=['x', 'y', 'z'])
df_objects = pd.DataFrame(obj_locations, columns=['x', 'y', 'z'])

df_camera["group"] = "Camera"
df_objects["group"] = pd.Series(obj_labels)
df_camera["size"] = 2
df_objects["size"] = 5
combined_df = pd.concat(
    [df_camera[['x', 'y', 'z', 'group', 'size']], 
     df_objects[['x', 'y', 'z', 'group', 'size']]], 
    ignore_index=True
)

fig = px.scatter_3d(combined_df,
                    x='x', 
                    y='z', 
                    z='y', 
                    color="group",
                    size = "size",ß
                    title="Camera Path and Object Spatial Locations",
                    width=800, height=800,
                    template='plotly_dark')
fig.show()